In [2]:
import os
import sys
sys.path.append('/home/longshen/work/AccGen/AccGen')
from models.vae_inference import PhraseVAE, BarVAE
import torch

data = torch.load('/data1/longshen/Datasets/Piano/POP909/latents/song_level_bar_seq_with_annot/dataset.pt')

ae = BarVAE().cuda()
song1_latent = data['001']['latent'].cuda()
song1_len = data['001']['length']
print("Song 1 latent shape:", song1_latent.shape)
print("Song 1 length:", song1_len)
recon = ae.decode_song(song1_latent)
print(recon)
print(recon[-20:])

from remi_z import MultiTrack
mt = MultiTrack.from_remiz_str(recon)
save_dir = '/data1/longshen/Results/AccGenResults/test_outputs/dataset/barldm_dataset_latent'
os.makedirs(save_dir, exist_ok=True)
mt.to_midi(os.path.join(save_dir, 'reconstructed_song1.mid'))

Song 1 latent shape: torch.Size([72, 512])
Song 1 length: 72
i-25 o-0 p-69 d-6 o-6 p-67 d-3 o-12 p-65 d-3 o-18 p-74 d-3 o-21 p-76 d-3 o-24 p-74 d-15 o-42 p-60 d-3 i-0 o-0 p-41 d-18 o-3 p-48 d-15 o-6 p-53 d-9 o-9 p-60 d-15 o-24 p-43 d-15 o-27 p-50 d-12 o-30 p-55 d-12 o-33 p-59 d-9 b-1 i-25 o-0 p-69 d-6 o-6 p-67 d-9 o-18 p-64 d-18 o-42 p-67 d-3 i-0 o-0 p-40 d-21 o-3 p-47 d-15 o-6 p-52 d-18 o-9 p-55 d-12 o-24 p-45 d-6 o-27 p-48 d-15 o-30 p-52 d-15 o-33 p-60 d-12 b-1 i-25 o-0 p-72 d-3 o-3 p-74 d-3 o-6 p-76 d-12 o-18 p-67 d-3 o-24 p-72 d-3 o-27 p-74 d-3 o-30 p-76 d-9 i-0 o-0 p-41 d-15 o-3 p-48 d-12 o-6 p-53 d-12 o-9 p-57 d-12 o-24 p-55 d-9 p-50 d-3 p-43 d-18 o-30 p-50 d-3 o-33 p-59 d-6 b-1 i-25 o-6 p-60 d-6 o-12 p-69 d-6 o-18 p-67 d-21 i-13 o-36 p-55 d-3 o-39 p-57 d-3 o-42 p-60 d-3 o-45 p-62 d-3 i-0 o-0 p-36 d-21 o-3 p-43 d-18 o-6 p-48 d-15 o-9 p-55 d-12 o-18 p-52 d-3 o-24 p-60 d-9 p-55 d-6 p-52 d-6 p-48 d-9 p-36 d-15 o-30 p-43 d-9 o-36 p-55 d-6 p-52 d-3 p-48 d-3 b-1 i-25 o-3 p-72 d-3 o-6 p

In [4]:
# Seems to be problematic. Try encode and decode with VAE
midi_fp = '/data1/longshen/Datasets/Piano/POP909/pop909_longshen/data_key_normed/001/001.mid'
original_mt = MultiTrack.from_midi(midi_fp)
original_str = original_mt.to_remiz_str()
print('original_str length:', len(original_str))
print(original_str)
latents = ae.encode_mt(original_mt, do_sample=True)
print('latents shape:', latents.shape)

recon_str = ae.decode_song(latents=latents)
print('recon_str length:', len(recon_str))
print(recon_str)

original_seq = original_str.split()
recon_seq = recon_str.split()
errors = 0
for i in range(min(len(original_seq), len(recon_seq))):
    if original_seq[i] != recon_seq[i]:
        print(f"Difference at token {i}: original '{original_seq[i]}', reconstructed '{recon_seq[i]}'")
        errors += 1
print(f"Total differences: {errors} out of {min(len(original_seq), len(recon_seq))} tokens.")

recon_mt = MultiTrack.from_remiz_str(recon_str)
recon_mt.to_midi(os.path.join(save_dir, 'reconstructed_song1_from_vae.mid'))

original_str length: 20542
i-25 o-0 p-69 d-6 o-6 p-67 d-3 o-12 p-65 d-3 o-18 p-74 d-3 o-21 p-76 d-3 o-24 p-74 d-15 o-42 p-60 d-3 i-0 o-0 p-41 d-18 o-3 p-48 d-15 o-6 p-53 d-9 o-9 p-60 d-15 o-24 p-43 d-15 o-27 p-50 d-12 o-30 p-55 d-12 o-33 p-59 d-9 b-1 i-25 o-0 p-69 d-6 o-6 p-67 d-9 o-18 p-64 d-18 o-42 p-67 d-3 i-0 o-0 p-40 d-21 o-3 p-47 d-15 o-6 p-52 d-18 o-9 p-55 d-12 o-24 p-45 d-6 o-27 p-48 d-15 o-30 p-52 d-15 o-33 p-60 d-12 b-1 i-25 o-0 p-72 d-3 o-3 p-74 d-3 o-6 p-76 d-12 o-18 p-67 d-3 o-24 p-72 d-3 o-27 p-74 d-3 o-30 p-76 d-9 i-0 o-0 p-41 d-15 o-3 p-48 d-12 o-6 p-53 d-12 o-9 p-57 d-12 o-24 p-55 d-9 p-50 d-3 p-43 d-18 o-30 p-50 d-3 o-33 p-59 d-6 b-1 i-25 o-6 p-60 d-6 o-12 p-69 d-6 o-18 p-67 d-21 i-13 o-36 p-55 d-3 o-39 p-57 d-3 o-42 p-60 d-3 o-45 p-62 d-3 i-0 o-0 p-36 d-21 o-3 p-43 d-18 o-6 p-48 d-15 o-9 p-55 d-12 o-18 p-52 d-3 o-24 p-60 d-9 p-55 d-6 p-52 d-6 p-48 d-9 p-36 d-15 o-30 p-43 d-9 o-36 p-55 d-6 p-52 d-3 p-48 d-3 b-1 i-25 o-3 p-72 d-3 o-6 p-74 d-3 o-9 p-76 d-3 o-30 p-74 d-3

In [ ]:
from experiments.autoencoders.lit_aes import (
    calculate_metrics_for_prolls,
    log_piano_roll,
    calculate_f1_ipod,
    compute_barvae_f1_metrics,
)
metrics = compute_barvae_f1_metrics(
    out_texts=[recon_str],
    tgt_texts=[original_str],
)
print("Reconstruction metrics:", metrics)

Reconstruction metrics: {'f1_op': 1.0, 'f1_opd': 1.0, 'f1_iopd': 1.0}
